In [ ]:
# main.py

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from gcngru import GRU_Model  # Import GRU_Model from gcngru.py
from gtmp import Main_Model  # Import Main_Model from gtmp.py
from training_data import TrainingData  # Make sure this is available or adapt as needed

# Define the dataset class
class EEGDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        signal_processed, Adj_dist_matrix, bdi_label = self.data[idx]
        return signal_processed, Adj_dist_matrix, bdi_label

def load_processed_data(file_path):
    return np.load(file_path, allow_pickle=True)

def prepare_data(data):
    X = TrainingData.trainingData_iterator(data)
    return X

def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for input_seq, adjacency_matrix, bdi_label in dataloader:
        input_seq = input_seq.to(device).float()
        adjacency_matrix = adjacency_matrix.to(device).float()
        bdi_label = bdi_label.to(device).float()

        optimizer.zero_grad()
        output = model(input_seq, adjacency_matrix)
        loss = criterion(output, bdi_label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Training Loss: {total_loss / len(dataloader)}')

def main():
    # Load processed signals
    processed_signals_path = 'processed_signals.npy'
    training_data = load_processed_data(processed_signals_path)

    # Prepare data for training
    data = prepare_data(training_data)
    dataset = EEGDataset(data)

    # Dataset and DataLoader
    batch_size = 40
    train_test_split_ratio = 0.7
    train_size = int(len(dataset) * train_test_split_ratio)
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model, loss function, optimizer
    nodes_dim = 16
    node_features_dim = 16
    hidden_dim = 64
    num_layers = 2
    dropout_prob = 0.5
    seq_len = 100  # Define your sequence length
    batch_size = 40
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = Main_Model(nodes_dim, node_features_dim, hidden_dim, num_layers, dropout_prob, seq_len, batch_size).to(device)
    criterion = nn.MSELoss()  # Adjust if needed
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        train(model, train_dataloader, criterion, optimizer, device)

    # Evaluate on test set
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for input_seq, adjacency_matrix, bdi_label in test_dataloader:
            input_seq = input_seq.to(device).float()
            adjacency_matrix = adjacency_matrix.to(device).float()
            bdi_label = bdi_label.to(device).float()

            output = model(input_seq, adjacency_matrix)
            predictions = torch.argmax(output, dim=1)
            total_correct += (predictions == bdi_label).sum().item()
            total_samples += bdi_label.size(0)

    print(f'Test Accuracy: {total_correct / total_samples * 100:.2f}%')

if __name__ == "__main__":
    main()
